In [56]:
import numpy as np
import skimage as sk
import cv2
import copy
from scipy.ndimage import gaussian_filter

In [61]:
def poolAndMax3d(input, pool=(2,2,2), v=''):
    v = v.count('+')
    # Make the 3D stack binnable by the size
    while input.shape[0] % pool[0] != 0: # If the image Y dimension is not even
        zero_plane = np.zeros_like(input[:pool[0] - input.shape[0] % pool[0], :, :]) # Create a 2D plane that matches the size of the data
        input = np.concatenate([zero_plane, input], axis=0) # Concatenate the zero-filled plane with the original data
        print(f"Modify Z: {input.shape}, added layer.") if v > 0 else None
    while input.shape[1] % pool[1] != 0: # If the image X dimension is not even
        zero_plane = np.zeros_like(input[:, :pool[1] - input.shape[1] % pool[1], :]) # Create a 2D plane that matches the size of the data
        input = np.concatenate([zero_plane, input], axis=1) # Concatenate the zero-filled plane with the original data
        print(f"Modify Y: {input.shape}, added layer.") if v > 0 else None
    while  input.shape[2] % pool[2] != 0: # If the image X dimension is not even
        zero_plane = np.zeros_like(input[:, :, :pool[2] - input.shape[2] % pool[2]]) # Create a 2D plane that matches the size of the data
        input = np.concatenate([zero_plane, input], axis=2) # Concatenate the zero-filled plane with the original data
        print(f"Modify X: {input.shape}, added layer.") if v > 0 else None

    print(input.shape) if v > 0 else None
    
    shapeZ, shapeY, shapeX = input.shape[0], input.shape[1], input.shape[2] # Compute the shape (new shape)
    newZ, newY, newX = int(shapeZ/pool[0]), int(shapeY/pool[1]), int(shapeX/pool[2]) # The new dimensions that the image will have
    # print(shapeY, shapeX, newY, newX)
    poolList = []
    maxList = [[[0 for _ in range(newX)] for _ in range(newY)] for _ in range(newZ)]
    print(f"Convolution from {shapeZ} {shapeY} {shapeX} to {len(maxList)} {len(maxList[0])} {len(maxList[0][0])}") if v > 0 else None

    c = 1
    total = int(newZ*newY*newX)
    for z in range(0, shapeZ, pool[0]):
        for y in range(0, shapeY, pool[1]):
            for x in range(0, shapeX, pool[2]):
                patch = input[z:z+pool[0], y:y+pool[1], x:x+pool[2]]
                poolList.append(patch.max())
                
                indices = np.unravel_index(np.argmax(patch), patch.shape)
                maxList[int(z / pool[0])][int(y / pool[1])][int(x / pool[2])] = (z + indices[0], y + indices[1], x + indices[2])
                
                print(f"{c}/{total}", end="\r") if v > 1 else None
                c+=1
                
    print(f"{total}/{total}, DONE!") if v > 1 else None
            
    poolArray = np.array(poolList)
    poolArray = np.reshape(poolArray, (newZ, newY, newX))
    maxList = np.array(maxList)
    
    return poolArray, maxList, pool



def deconv3d(input, maxList, pool=(2,2,2), v=''):
    v = v.count('+')
    shapeZ, shapeY, shapeX = maxList.shape[0], maxList.shape[1], maxList.shape[2]
    newZ, newY, newX = int(maxList.shape[0]*pool[0]), int(maxList.shape[1]*pool[1]), int(maxList.shape[2]*pool[2])
    print(f"Deconvolution from {shapeZ} {shapeY} {shapeX} to {newZ} {newY} {newX}") if v > 0 else None
    
    output = [[[0 for _ in range(newX)] for _ in range(newY)] for _ in range(newZ)]
    
    c = 1
    total = int(shapeZ*shapeY*shapeX)
    for z in range(0, shapeZ):
        for y in range(0, shapeY):
            for x in range(0, shapeX):
                indices = maxList[z, y, x]
                output[indices[0]][indices[1]][indices[2]] = input[z, y, x]
                
                print(f"{c}/{total}", end="\r") if v > 1 else None
                c+=1
                
    print(f"{total}/{total}, DONE!") if v > 1 else None
    
    output = np.array(output)

    return output



In [58]:
def conv_deconv3D(im3d, times=3, v=''):
    v = v.count('+')
    output = []
    
    pool = [0 for _ in range(times)]
    maxList = [0 for _ in range(times)]
    deconv = [0 for _ in range(times)]
    
    c = 0
    for t in range(times): # CONVOLUTION
        print(f"Convolving: {c+1}/{times}") if v > 0 else None
        if c == 0:
            pool_n, maxList_n, _ = poolAndMax3d(im3d)
        else:
            pool_n, maxList_n, _ = poolAndMax3d(pool[c-1])
            
        pool[t] = pool_n
        maxList[t] = maxList_n
        c+=1
        
        
    
    
    # MAKE THE MASK
    #print(pool[c-1].shape)
    lastPool = gaussian_filter(pool[c-1], sigma=2)
    #print(lastPool.shape)
    mean = lastPool.mean()
    pooled_mask = copy.copy(lastPool)
    pooled_mask[lastPool > mean] = 255
    pooled_mask[lastPool <= mean] = 0
    
    
    c = times
    for t in range(times): # UPCONVOLUTION
        print(f"Deconvolving: {c}/{times}") if v > 0 else None
        if c == times:
            deconv_n = deconv3d(pooled_mask, maxList[c-1])
        else:
            deconv_n = deconv3d(deconv[times-c-1], maxList[c-1])
            
        deconv[t] = deconv_n
        c-=1

    
    output = np.array(deconv_n)
    return output

In [62]:
image = sk.io.imread('./domes/Dome103_processed_n2v2d.tif')
points = conv_deconv3D(image, times=3)

Convolving: 1/3
Convolving: 2/3
Convolving: 3/3
Deconvolving: 3/3
Deconvolving: 2/3
Deconvolving: 1/3


In [2]:
import napari

viewer = napari.Viewer()
viewer.add_image(points)

NameError: name 'points' is not defined

In [68]:
pointsXoriginal = points*image
coordinates = np.transpose(np.nonzero(pointsXoriginal))
intensity_column = [image[idx[0], idx[1], idx[2]] for idx in coordinates]
pointsXoriginal = np.column_stack((coordinates, intensity_column))


In [76]:
import tifffile as tiff
tiff.imsave('./dottedIntensity.tiff', points*image)


C:\Users\SPIM\AppData\Local\Temp\ipykernel_632\790074462.py:2: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave('./dottedIntensity.tiff', points*image)


In [75]:
print(pointsXoriginal.shape, pointsXoriginal[:5])

np.savetxt('pointsIntensity.txt', pointsXoriginal, fmt='%d %d %d %.2f', header='z y x intensity', comments='')

(102146, 4) [[ 54.         344.         459.          32.93125916]
 [ 54.         348.         480.          29.84007263]
 [ 55.         331.         489.          30.28404808]
 [ 55.         333.         487.          29.67397881]
 [ 55.         334.         472.          32.48284912]]
